## Reed-Muller Code:

Status: Currently, it only works for r = 1.

In [1]:
import numpy as np
import itertools

class ReedMullerCode:
    
    def __init__(self, r, m):
            
        self.r = r
        self.m = m
        self.q = 2
        
        if r > 1:
            raise ValueError('Currently, only r = 1 is supported')
        
        self.n = 2**self.m
        self.k = sum(binomial(self.m,i) for i in range(self.r + 1))
        self.d = 2**(self.m - self.r)
        
        # Initializing field
        self.F = FiniteField(self.q)
        
        self.G = matrix(self.F, self.k, self.n, self.generateG(self.r, self.m))
        
        
    def generateG(self, r, m):
        ones = np.repeat(1, 2**m)
        bases = [np.array([(i // 2**a) % 2 for i in range(2**m)]) for a in range(m)]
        if r == 1:
            G = np.concatenate([ones.reshape(1,2**m), np.stack(bases)])
            return G
        elif r > 1:
            others = []
            for i in range(2,m-1):
                others.extend([math.prod(x) for x in itertools.combinations(bases, i)])
            
            G = np.concatenate([ones.reshape(1,2**m), np.stack(bases), np.stack(others)])
            return G
        else:
            raise ValueError('wrong value of r')
    
    
    def Encoding(self, message, zeropad = True):
        
        rem = len(message) % self.k
        
        if rem != 0:
            if zeropad:
                message.extend([self.F(0)]*(self.k-rem))
            else:
                raise ValueError('k does not divide input size')
                
                
        c = []
        
        # Encoding each chunk of size k
        for i in range(0, len(message), self.k):
            c.extend(self.EncodeChunk(message[i:i+self.k]))
        
        return vector(self.F, c)
            
    def EncodeChunk(self, chunk):
        
        # Encode a chunk of size k
        if len(chunk) != self.k:
            raise ValueError('Invalid chunk size')
            
        c = vector(self.F, chunk) * self.G

        return c
        
    
    def Decoding(self, received):
        
        # Check input size
        if len(received) % self.n != 0:
            raise ValueError('Invalid input size')
            
        c = []
        
        for i in range(0,len(received),self.n):
            c.extend(self.DecodeChunk(received[i:i+self.n]))
            
        return vector(self.F, c)
                
            
    def DecodeChunk(self, word):
        # Reed Decoding algorithm
        
        if (len(word) != self.n):
            raise ValueError('Invalid chunk size')
        
        decoded = []
        
        if r == 2:
            variables = [i for i in range(1, self.m + 1)]
            combinations = []
            combinations.append(list(itertools.combinations(variables,2)))
            combinations = list(combinations)
            
            for i in range(factorial(self.m)-1, -1, -1):
                
                # compute the complementary set T
                T = [i for i in range(1,self.m+1)]
                for i in range(self.r):
                    T.remove(combinations[0][i])
                    
                # compute the current set S
                S = [1] * self.m
                for i in range(len(T)):
                    S[T[i] - 1] = 0
                S = vector(self.F, S)
                
                # compute all combinations of the variables in the complementary set T
                combinations = []
                for i in range(len(T)+1):
                    combinations.append(list(itertools.combinations(T,i)))
                combinations = list(combinations)
                    
            
        
        # first decode a_1, ..., a_m, in reverse order (a_m, ..., a_1) 
        for i in range(self.m,0,-1):
            
            # compute the complementary set T 
            T = [i for i in range(1,self.m+1)]
            del T[i-1]    
            
            # compute the current set S
            S = [1] * self.m
            for i in range(len(T)):
                S[T[i] - 1] = 0
            S = vector(self.F, S)

            # compute all combinations of the variables in the complementary set T
            combinations = []
            for i in range(len(T)+1):
                combinations.append(list(itertools.combinations(T,i)))
            combinations = list(combinations)
            
            # compute all the votings
            votes = []
            for i in range(len(combinations)):
                for j in range(len(combinations[i])):
                    voting_set = [0] * self.m
                    for k in range(len(combinations[i][j])):
                        voting_set[combinations[i][j][k] - 1] = 1
                    voting_set = vector(self.F, voting_set)
                    votes.append(word[ZZ(voting_set.list(),2)] + word[ZZ((voting_set+S).list(),2)])
            
            # decode coefficient by comparing number of ones with number of zeros in the list of votings
            decoded = self.MajorityDecoding(votes) + decoded
        
        # decode a_0 by subtracting the part with a_1, ..., a_m
        self.G_part = matrix(self.F, self.k - 1, self.n, self.G[1:][:])
        word_part = vector(self.F, decoded) * self.G_part
        decoded = self.MajorityDecoding((word+word_part).list()) + decoded
        
        return decoded
    
            
            
    def MajorityDecoding(self, word):
        # Decode by comparing the number of ones with the number of zeros
        
        if word.count(1) >= floor(len(word) / 2):
            return [1]
        elif word.count(1) < floor(len(word) / 2):
            return [0]
        else:
            return "decoding failure"

In [2]:
C = ReedMullerCode(r = 1, m = 4)
C.G

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1]
[0 0 1 1 0 0 1 1 0 0 1 1 0 0 1 1]
[0 0 0 0 1 1 1 1 0 0 0 0 1 1 1 1]
[0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1]

In [3]:
m = vector(GF(2), [0,1,0,0,0])
c = C.Encoding(m)
C.Decoding(c)

(0, 1, 0, 0, 0)